<a href="https://colab.research.google.com/github/jodfedlet/TCC/blob/main/TCC/Twitter/LDA_with_gensim_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Objetivos
 - Dividir o df por mês
 - Para cada subdf, realizar a experimentação:
    - definir numeros de tópicos
    - criar o modelo LDA
        - passar o número de tópico a cada vez
        - definir o valor de alfa e beta
    - Aplicar o modelo LDA para cada número de tópicos
    - Encontrar o número de tópicos que melhor representa os documentos (fase que pode ser realizada à partir de conhecimentos empíricos dos documentos ou por métrica de coerência)
 - Salvar o melhor número de tópicos para cada subdf   

In [1]:
!pip install gensim
!pip install pyldavis
! pip install bs4
! pip install contractions
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:

import pandas as pd
import re
import json
import gzip
import gensim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import pyLDAvis.gensim_models
# import pickle 
# import pyLDAvis

from google.colab import drive
drive.mount("/content/drive")

from pprint import pprint
from gensim import corpora
from bs4 import BeautifulSoup
from gensim.models import Phrases, LdaModel, LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
try:
 dftweets = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/pre_processed_tweets.csv', lineterminator='\n', index_col=0)
except:
  print("Não foi possível carregar o arquivo")

In [4]:
dftweets.head()

,datetime,tweet_id,user_id,text,count_word,text_preprocessed,bigrams_text,text_lematizados
1,2022-05-30 23:00:53+00:00,1531410348141592577,1473798922472767488,@BetoLuiz_RU @jonesmanoel_PCB @deccache O Hait...,15,haiti desgraca fato erros anulam acertos fala ...,haiti desgraca fato erros anulam acertos fala ...,haiti desgraca fato erro anular acerto falir s...
2,2022-05-30 22:57:20+00:00,1531409451122577410,1521912766151417856,O senhor vai investir na educação da Venezuela...,15,senhor vai investir educacao venezuela argenti...,senhor vai investir_educacao venezuela argenti...,senhor ir investir_educacao venezuela argentin...
3,2022-05-30 22:55:14+00:00,1531408925945434112,1253882714723409920,"@rsallesmma Que nada, na mão de vocês já tá vi...",7,nada mao voces virando mistura deserto saara h...,nada mao voces virando mistura deserto_saara h...,nado mao voces virar misturar deserto_saara haiti
4,2022-05-30 22:55:14+00:00,1531408923277807617,40320768,@JoppertLeonardo @jonesmanoel_PCB @deccache En...,21,entramos brics invadimos haiti mando bush lula...,entramos brics invadimos haiti mando_bush lula...,entrar brics invadir haiti mando_bush lula diz...
5,2022-05-30 22:37:49+00:00,1531404543568781313,1082697350441824257,"@alice_mader9 kkkkkk e era 2 haitiano, n enten...",3,haitiano entendi nada falavam,haitiano entendi_nada falavam,haitiano entendi_nada falar


In [5]:
dftweets.bigrams_text.head().tolist()

['haiti desgraca fato erros anulam acertos fala sobre guerra_ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir_educacao venezuela argentina haiti cuba etc paises_comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto_saara haiti',
 'entramos brics invadimos haiti mando_bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia_china inimigos mortais vai funcionar',
 'haitiano entendi_nada falavam']

In [13]:
#agrupar df por mês e adicionar os períodos numa lista
dftweets['datetime'] = pd.to_datetime(dftweets.datetime, format='%Y-%m-%d %H:%M:%S')
df_by_year_month = [df_year_month for df_year_month in dftweets.groupby([dftweets['datetime'].dt.year.rename('Year'), dftweets['datetime'].dt.month.rename('Month')])]

In [14]:
#2018-01 até 2022-05
len(df_by_year_month)

53

In [8]:
def get_tweets_period(df_tuple):
  #idenficar o peródo dos tweets
  m = df_tuple[0][1]
  month = '0'+ str(m) if m < 10 else m
  return str(df_tuple[0][0]) +'-'+str(month)

###gensim oficial doc 
https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#sphx-glr-auto-examples-tutorials-run-lda-py

In [113]:
k_topicos = [3, 5, 7, 10, 15, 25]

In [123]:
for idx, df_y_m in enumerate(df_by_year_month):

  if idx == 3: break #será removido depois

  #df period and mini stats
  period = get_tweets_period(df_y_m)
  print(f'Período: {period}')
  words_count = df_y_m[1]['count_word'].sum()
  docs = df_y_m[1].text_lematizados.tolist()
  tam_doc = len(docs)
  # print(f'Tamanho do documento: {tam_doc} tweets, {words_count} palavras e {round(words_count/tam_doc, 2)} de médias de palavras')
  # print('-'*80)

  # print(docs)
  # break

  #palavras que não são significativas
  stop_words = ['nao','para','pra','aqui','trump', 'ir', 'ter', 'esse', 'todo','tudo', 'este', 'ser', 'querer', 'sao', 'pai', 'pro', 'ver', 'voce'] # 'fazer', 'dizer', 'falar'

  texts = [[word for word in doc.split() if word not in stop_words and len(word) > 1] for doc in docs]
 
  #texts = docs_list[0:3]

  #criando o dicionário da representação dos documentos
  dictionary = corpora.Dictionary(texts)

  #Filtre as palavras que ocorrem em menos de 20 documentos ou em mais de 50% dos documentos.
  dictionary.filter_extremes(no_below=int(tam_doc * 0.015), no_above=0.5)

  #criando o corpus (bags-of-words dos documentos)
  corpus = [dictionary.doc2bow(text) for text in texts]

  # print(texts[0])
  # print(corpus[0])

  # print(f'Number of unique tokens: {len(dictionary)}')
  # print(f'Number of documents: {len(corpus)}')
 
  for k_topico in k_topicos:

    print(f'Para {k_topico} tópicos -------------')

    LDA_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=k_topico, chunksize=tam_doc, 
                         alpha='auto', eta='auto', iterations=1000, random_state=100, passes=100, eval_every=None)
    top_topics = LDA_model.top_topics(corpus)

    # Compute Perplexity
    print('\nPerplexity: ', LDA_model.log_perplexity(corpus))
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=LDA_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)
  
    #pprint(top_topics)
    #pprint(LDA_model.print_topics())

    for i, topic in enumerate(top_topics):
      topic_list = [word[1] for word in topic[0]]
      print(f'{i} : {topic_list}')
    
    print('*'*200)
    print()
    #break

  print()
  #break

  

Período: 2018-01
Para 3 tópicos -------------

Perplexity:  -4.127386191384148

Coherence Score:  0.30328902537139446
0 : ['merda', 'sobrar', 'paises', 'presidente', 'chamar', 'salvador', 'paises_africanos', 'donald_trump', 'racista', 'usar', 'imigrante', 'dizer', 'referir', 'africano', 'estados_unidos', 'tambem', 'americano', 'buraco_merda', 'nacoes_africanas', 'buracos_merda']
1 : ['haitiano', 'falar', 'caro', 'dizer', 'ano', 'saber', 'achar', 'vir', 'povo', 'ficar', 'poder', 'pessoa', 'ainda', 'dar', 'brasileiro', 'dia', 'outro', 'falir', 'mundo', 'onde']
2 : ['brasil', 'africa', 'eua', 'fazer', 'dizer', 'outros_paises', 'gente', 'agora', 'trump_chama', 'america_central', 'contra', 'entao', 'atar', 'melhor', 'bem', 'buracos_merda', 'onu', 'conhecer', 'novo', 'historiar']
********************************************************************************************************************************************************************************************************

Para 5 tópico